In [11]:
import lsst.daf.base as dafBase
import lsst.daf.persistence as dafPersist
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import Utils.DiaSourceTools as DSTools

In [12]:
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/output"
butler = dafPersist.Butler(DATADIR)

In [13]:
def multi_match_catalogs(catalog_list, data_refs):
    multi_matches = None

    for catalog, (visit, ccd) in zip(catalog_list, data_refs):

            if multi_matches is None and len(catalog)>0:
                multi_matches = afwTable.MultiMatch(catalog[0].schema, {'visit':int, 'ccd':int}, radius=afwGeom.Angle(1./3600., afwGeom.degrees))
            if multi_matches is not None:
                multi_matches.add(catalog, {'visit':int(visit), 'ccd':int(ccd)})

    results = multi_matches.finish(removeAmbiguous=False)  
    return results

In [14]:
data_refs = butler.queryMetadata("deepDiff_class_diaSrc", format=['visit', 'ccd'], dataId={'filter':'r'})
def_data_refs = []
catalogs = []
for data_ref in data_refs:
    if butler.datasetExists("deepDiff_class_diaSrc", dataId={'visit':data_ref[0], 'ccd':data_ref[1], 'filter':'r'}):
        catalog = butler.get("deepDiff_class_diaSrc", dataId={'visit':data_ref[0], 'ccd':data_ref[1], 'filter':'r'})
        catalogs.append(catalog)
        def_data_refs.append(data_ref)
        


In [ ]:
results = multi_match_catalogs(catalogs, def_data_refs)

In [ ]:
len(results)

In [ ]:
light_curves=DSTools.get_light_curves_from_multimatch_results2(results)
print len(light_curves)

In [ ]:
lcs = []
for t_light_curve in light_curves:
    lcs.append(DSTools.build_lightcurve(t_light_curve, "base_CircularApertureFlux_3_0_flux"))

In [ ]:

import re
import astropy.coordinates as coord
import astropy.units as u

filter='r'

f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table9.dat.txt','r')
data_elems = f.read()
elems = re.findall('^(06D3.*?)\\|(.*?)\\|', data_elems, re.MULTILINE)

f = open('/renoir_data_02/jpreyes/lsst_data/sn_control/J_A+A_523_A7_table10.dat.txt','r')
data = f.read()

snls_array = []
for sn in elems:
    c = coord.SkyCoord(sn[-1], unit=(u.hourangle, u.deg))
    
    m = re.findall('^'+str(sn[0])+'\\|('+filter+')\\|(.*?)\\|(.*?)\\|(.*?)$', data, re.MULTILINE)

    snls_lc = DSTools.build_light_curve_from_snls_file(m, c)
    plt.errorbar(snls_lc['mjd'], snls_lc['flux'], yerr=snls_lc['flux_error'], fmt='.', color='blue')
    snls_array.append(snls_lc)
    plt.show()

In [ ]:
matches = DSTools.match_control_group(lcs,snls_array)
float(len(matches))/float(len(snls_array))